In [ ]:
!pip install pyarrow -U

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

### Q3

In [ ]:
df_1 = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet", 
                       engine="pyarrow")

In [ ]:
print(len(df_1))
df_1.info()

### Q4

In [ ]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [ ]:
df = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet")

In [ ]:
df.info()

### Q5

In [ ]:
df.head()

In [ ]:
df = df[["PULocationID", "DOLocationID", "duration"]]
df[["PULocationID", "DOLocationID"]] = df[["PULocationID", "DOLocationID"]].astype(str)
train_dicts = df[["PULocationID", "DOLocationID"]].to_dict(orient='records')

In [ ]:
vectorizer = DictVectorizer()
vectorizer.fit(train_dicts)

In [ ]:
feature_mat = vectorizer.transform(train_dicts)
feature_mat.shape

In [ ]:
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(feature_mat, y_train)

y_pred = lr.predict(feature_mat)

root_mean_squared_error(y_train, y_pred)